In [3]:
import argparse
import boto3
import os, sys, time
from pathlib import Path

In [4]:
session = boto3.Session()

In [5]:
ec2 = session.resource('ec2')
name='fast-ai'

In [6]:
def get_vpc(name):
    vpcs = list(ec2.vpcs.filter(Filters=[{'Name': 'tag-value', 'Values': [name]}]))
    return vpcs[0] if vpcs else None

In [7]:
def get_vpc_ids(name):
    vpc = get_vpc(name)
    if vpc is None: return None
    sg = list(vpc.security_groups.all())[0]
    subnet = list(vpc.subnets.all())[0]
    return vpc.id, sg.id, subnet.id

In [14]:
vpc_id, sg_id, subnet_id = get_vpc_ids(name)

In [9]:
region2ami = {
    'us-west-2': 'ami-8c4288f4',
    'eu-west-1': 'ami-b93c9ec0',
    'us-east-1': 'ami-c6ac1cbc'
}

In [12]:
instance_type = 't2.nano'

In [13]:
ami = region2ami[session.region_name]

In [41]:
def create_instance(name):
    ami = region2ami[session.region_name]
    vpc_id, sg_id, subnet_id = get_vpc_ids(name)
    network_interfaces=[{
    'DeviceIndex': 0,
    'SubnetId': subnet_id,
    'Groups': [sg_id],
    'AssociatePublicIpAddress': True            
    }]
    block_device_mappings = [{ 
        'DeviceName': '/dev/sda1', 
        'Ebs': { 
            'VolumeSize': 128, 
            'VolumeType': 'gp2' 
        } 
    }]
    instance = ec2.create_instances(ImageId=ami, InstanceType=instance_type, 
                     MinCount=1, MaxCount=1,
                     KeyName=f'aws-key-{name}',
                     BlockDeviceMappings=block_device_mappings,
                     NetworkInterfaces=network_interfaces
                    )[0]
    instance.create_tags(Tags=[{'Key':'Name','Value':f'{name}-gpu-machine'}])
    
    print('Instance created...')
    instance.wait_until_running()
    print('Creating public IP address...')
    ec2c = session.client('ec2')
    alloc_addr = ec2c.allocate_address(Domain='vpc')
    ec2c.associate_address(InstanceId=instance.id, AllocationId=alloc_addr['AllocationId'])
    
    print('Rebooting...')
    instance.reboot()
    instance.wait_until_running()
    return instance

In [24]:
network_interfaces=[{
    'DeviceIndex': 0,
    'SubnetId': subnet_id,
    'Groups': [sg_id],
    'AssociatePublicIpAddress': True            
}]
block_device_mappings = [{ 
    'DeviceName': '/dev/sda1', 
    'Ebs': { 
        'VolumeSize': 128, 
        'VolumeType': 'gp2' 
    } 
}]

In [25]:
ec2.create_instances(ImageId=ami, InstanceType=instance_type, 
                     MinCount=1, MaxCount=1,
                     KeyName=f'aws-key-{name}',
                     BlockDeviceMappings=block_device_mappings,
                     NetworkInterfaces=network_interfaces
                    )

[ec2.Instance(id='i-01ce7c7c9dc8f8a3d')]

In [26]:
instance = list(ec2.instances.all())[0]

In [27]:
instance.create_tags(Tags=[{'Key':'Name','Value':f'{name}-gpu-machine'}])

[ec2.Tag(resource_id='i-01ce7c7c9dc8f8a3d', key='Name', value='fast-ai-gpu-machine')]

In [28]:
vpc = instance.vpc

In [30]:
ec2c = session.client('ec2')


In [32]:
alloc_addr = ec2c.allocate_address(Domain='vpc')

In [33]:
alloc_addr

{'AllocationId': 'eipalloc-164bec2a',
 'Domain': 'vpc',
 'PublicIp': '52.43.199.214',
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 28 Mar 2018 20:26:17 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'c75ca463-7769-482b-92b3-7c2d0d01f847',
  'RetryAttempts': 0}}

In [34]:
ec2c.associate_address(InstanceId=instance.id, AllocationId=alloc_addr['AllocationId'])

{'AssociationId': 'eipassoc-ae0eb352',
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 28 Mar 2018 20:28:29 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '8e6c2a8f-1e01-4d2f-bf3c-1257bd78140a',
  'RetryAttempts': 0}}

In [35]:
instance.wait_until_running()

In [36]:
instance_url = instance.public_ip_address

In [37]:
instance_url

'34.217.174.189'

In [39]:
instance.reboot()

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 28 Mar 2018 20:30:05 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'f27d1320-a5cf-4994-8c4f-46e8dd94c02a',
  'RetryAttempts': 0}}

In [ ]:

export instanceId=$(aws ec2 run-instances --image-id $ami --count 1 --instance-type $instanceType --key-name aws-key-$name --security-group-ids $securityGroupId --subnet-id $subnetId --associate-public-ip-address --block-device-mapping "[ { \"DeviceName\": \"/dev/sda1\", \"Ebs\": { \"VolumeSize\": 128, \"VolumeType\": \"gp2\" } } ]" --query 'Instances[0].InstanceId' --output text)
aws ec2 create-tags --resources $instanceId --tags --tags Key=Name,Value=$name-gpu-machine
export allocAddr=$(aws ec2 allocate-address --domain vpc --query 'AllocationId' --output text)

echo Waiting for instance start...
aws ec2 wait instance-running --instance-ids $instanceId
sleep 10 # wait for ssh service to start running too
export assocId=$(aws ec2 associate-address --instance-id $instanceId --allocation-id $allocAddr --query 'AssociationId' --output text)
export instanceUrl=$(aws ec2 describe-instances --instance-ids $instanceId --query 'Reservations[0].Instances[0].PublicDnsName' --output text)
#export ebsVolume=$(aws ec2 describe-instance-attribute --instance-id $instanceId --attribute  blockDeviceMapping  --query BlockDeviceMappings[0].Ebs.VolumeId --output text)

# reboot instance, because I was getting "Failed to initialize NVML: Driver/library version mismatch"
# error when running the nvidia-smi command
# see also http://forums.fast.ai/t/no-cuda-capable-device-is-detected/168/13
aws ec2 reboot-instances --instance-ids $instanceId